In [2]:
!pip install -q openai==1.2.0 \tiktoken==0.6.0 \
pypdf==4.0.1 \
langchain==0.1.1 \
langchain-community==0.0.13 \
chromadb==0.4.22 \
sentence-transformers==2.3.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 2.4 MB/s eta 0:00:0

In [3]:
import json
import tiktoken
import pandas as pd
from openai import AzureOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings.sentence_transformer import (SentenceTransformerEmbeddings)
from langchain_community.vectorstores import Chroma


In [6]:
with open('/content/config.json', 'r') as az_creds:
 data = az_creds.read()

creds = json.loads(data)

client = AzureOpenAI(
 azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
 api_key=creds["AZURE_OPENAI_KEY"],
 api_version=creds["AZURE_OPENAI_APIVERSION"]
)

deployment_name = creds["CHATGPT_MODEL"]
print(deployment_name)

msft-June2024


In [11]:
!unzip /content/tesla-annual-reports.zip

Archive:  /content/tesla-annual-reports.zip
   creating: tesla-annual-reports/
  inflating: tesla-annual-reports/tsla-20221231-gen.pdf  
  inflating: __MACOSX/tesla-annual-reports/._tsla-20221231-gen.pdf  
  inflating: tesla-annual-reports/tsla-20231231-gen.pdf  
  inflating: __MACOSX/tesla-annual-reports/._tsla-20231231-gen.pdf  
  inflating: tesla-annual-reports/tsla-10ka_20211231-gen.pdf  
  inflating: __MACOSX/tesla-annual-reports/._tsla-10ka_20211231-gen.pdf  
  inflating: tesla-annual-reports/tsla-10k_20201231-gen.pdf  
  inflating: __MACOSX/tesla-annual-reports/._tsla-10k_20201231-gen.pdf  
  inflating: tesla-annual-reports/tsla-10k_20191231-gen_0.pdf  
  inflating: __MACOSX/tesla-annual-reports/._tsla-10k_20191231-gen_0.pdf  


In [12]:
pdf_folder_location = "/content/tesla-annual-reports"
pdf_loader = PyPDFDirectoryLoader(pdf_folder_location)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)



In [13]:
# Takes 5 minutes to run
tesla_10k_chunks_ada = pdf_loader.load_and_split(text_splitter)
len(tesla_10k_chunks_ada)
print (f'You have {len(tesla_10k_chunks_ada)} document(s) in your data')
print (f'There are {len(tesla_10k_chunks_ada[0].page_content)} characters in  document')
print (f'Here is a sample: {tesla_10k_chunks_ada[0].page_content[:200]}')

/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


You have 3342 document(s) in your data
There are 1283 characters in  document
Here is a sample: UNITED	STATES
SECURITIES	AND	EXCHANGE	COMMISSION
Washington,	D.C.	20549
FORM	
10-K
(Mark	One)
x
ANNUAL	REPORT	PURSUANT	TO	SECTION	13	OR	15(d)	OF	THE	SECURITIES	EXCHANGE	ACT	OF	1934
For	the	fiscal	year


# New Section

In [14]:
# @title Store Word Embeddings in Vector Database
tesla_10k_collection_1 = 'tesla-10k-2019-to-2023'
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [15]:
# Takes some time to run
vectorstore = Chroma.from_documents(
 tesla_10k_chunks_ada,
 embedding_model,
 collection_name=tesla_10k_collection_1,
 persist_directory='/content/tesla_db'
)
vectorstore.persist()



In [16]:
vectorstore_persisted = Chroma(collection_name=tesla_10k_collection_1,
 persist_directory='/content/tesla_db',
 embedding_function=embedding_model
)


In [17]:
retriever = vectorstore_persisted.as_retriever(
 search_type='similarity',
 search_kwargs={'k': 5}
)


In [19]:
# @title Prompt Design
qna_system_message = """
You are an assistant to a financial services firm who answers user queries on annual reports.
User input will have the context required by you to answer user questions.
This context will begin with the token: ###Context.
The context contains references to specific portions of a document relevant to the user query.

User questions will begin with the token: ###Question.

Please answer user questions only using the context provided in the input.
Do not mention anything about the context in your final answer. Your response should only contain the answer to the question.

If the answer is not found in the context, respond "I don't know".
"""
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

In [20]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')
!gdown https://drive.google.com/uc?export=download&id=1hWbAWhJr5xsl0sAvvEq9Wpo8ItCdZpdq



Downloading...
From: https://drive.google.com/uc?export=download
To: /content/uc?export=download
1.69kB [00:00, 4.90MB/s]


In [21]:
# @title Retrieve relevant Chunk From Doc
user_input = "What was the annual revenue of the company in 2022?"
relevant_document_chunks = retriever.get_relevant_documents(user_input)
#print(len(relevant_document_chunks))
for document in relevant_document_chunks:
 print(document.page_content.replace("\t", " "))
 break


systems.
In 2020, we recognized total revenues of $31.54 billion, representing an increase of $6.96 billion compared to the prior year. We continue to ramp
production, build new manufacturing capacity and expand our operations to enable increased deliveries and deployments of our products and further revenue
growth.
In 2020, our net income attributable to common stockholders was $721 million, representing a favorable change of $1.58 billion compared to the prior
year. In 2020, our operating margin was 6.3%, representing a favorable change of 6.6% compared to the prior year. We continue to focus on operational
efficiencies, while we have seen an acceleration of non-cash stock-based compensation expense due to a rapid increase in our market capitalization and updates
to our business outlook.
We ended 2020 with $19.38 billion in cash and cash equivalents, representing an increase of $13.12 billion from the end of 2019. Our cash flows from
operating activities during 2020 was $5.94 billion

In [22]:
context_list = [d.page_content for d in relevant_document_chunks]
context_for_query = ". ".join(context_list)

prompt = [
 {'role':'system', 'content': qna_system_message},
 {'role': 'user', 'content': qna_user_message_template.format(
 context=context_for_query,
 question=user_input
 )
 }
]
try:
 response = client.chat.completions.create(
 model=deployment_name,
 messages=prompt,
 temperature=0
 )
 prediction = response.choices[0].message.content.strip()
except Exception as e:
 prediction = f'Sorry, I encountered the following error: \n {e}'

print(prediction)


The annual revenue of the company in 2022 was $81.462 billion.
